# 서울 자치구 주유소

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

## 데이터 찾기

In [2]:
# 드라이버로 링크 열기
url = 'https://www.opinet.co.kr/user/main/mainView.do'
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(2)

In [3]:
# '싼 주유소 찾기' 페이지 열기
driver.find_element(By.CSS_SELECTOR, '.m1').click()

In [4]:
# 강남구 주유소
driver.find_element(By.XPATH, '//*[@id="SIGUNGU_NM0"]/option[2]').click()
driver.find_element(By.ID, 'glopopd_excel').click()

In [7]:
# 서울 모든 자치구 주유소
for i in range(2, 27):
    driver.find_element(By.XPATH, f'//*[@id="SIGUNGU_NM0"]/option[{i}]').click()
    driver.find_element(By.ID, 'glopopd_excel').click()

In [14]:
# 첫번째 엑셀 데이터 프레임으로 변환
df = pd.read_excel('C:/Users/user/Downloads/지역_위치별(주유소).xls', header=2)

In [15]:
# 엑셀 파일 불러와서 데이터 프레임에 추가하기
for i in range(1, 25):
    df_temp = pd.read_excel(f'C:/Users/user/Downloads/지역_위치별(주유소) ({i}).xls', header=2)
    df = pd.concat([df, df_temp])

In [16]:
# 인덱스 지정
df.set_index([pd.Index(range(443))], inplace=True)

In [17]:
df = df.loc[:,['상호', '주소', '상표', '셀프여부', '휘발유']]

In [18]:
df.rename(columns = {'휘발유':'가격'}, inplace = True)

In [19]:
# 저렴한 주유소 Top5
df_cheap = df.sort_values(by=['가격']).head(5)
df_cheap

,상호,주소,상표,셀프여부,가격
60,화곡역주유소,서울 강서구 강서로 154 (화곡동),알뜰주유소,Y,1485
59,이케이에너지(주) 강서주유소,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,Y,1485
61,뉴신정주유소,서울 강서구 곰달래로 207 (화곡동),알뜰주유소,N,1498
207,마포시엠주유소,서울 마포구 월드컵북로 62,SK에너지,N,1507
208,성산대교셀프주유소,서울 마포구 성산로 144,현대오일뱅크,Y,1507


In [20]:
# 비싼 주유소 Top5
df_exp = df.sort_values(by=['가격']).tail(5)
df_exp

,상호,주소,상표,셀프여부,가격
31,(주)새서울네트웍스 제이제이주유소,서울 강남구 언주로 716,현대오일뱅크,N,2298
428,필동주유소,서울 중구 퇴계로 196 (필동2가),GS칼텍스,N,2419
32,(주)만정에너지 삼보주유소,서울 강남구 봉은사로 433 (삼성동),GS칼텍스,N,2578
395,서계주유소,서울 용산구 청파로 367 (청파동),GS칼텍스,N,2625
429,서남주유소,서울 중구 통일로 30,SK에너지,N,2631


## 지도 시각화

In [21]:
import folium
import json
from urllib.parse import quote

In [22]:
# api 키 불러오기
with open('카톡_api_key.txt') as f:
    api_key = f.read()

In [23]:
search_url = "https://dapi.kakao.com/v2/local/search/address.json"

### 지도 표시

In [24]:
# 지도
map = folium.Map(location=[37.581, 126.986], zoom_start=11)

for i in df.index:
    try:
        road_addr = df['주소'].loc[i]
        station_name = df['상호'].loc[i]
        station_type = df['상표'].loc[i]
        url = f'{search_url}?query={quote(road_addr)}'
        result = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}).json()
        lng = float(result['documents'][0]['x'])
        lat = float(result['documents'][0]['y'])
        folium.Marker(
            location = [lat, lng], 
            popup = folium.Popup(road_addr, max_width=200),
            tooltip = (station_name+f"({station_type})"), 
            icon = folium.Icon(color = 'green', icon = 'glyphicon glyphicon-tint')
        ).add_to(map)
    except:
        print(station_name)
title = "<h3 align = 'center' style = 'font-size:20px'> 서울자치구 주유소 위치</h3>"
map.get_root().html.add_child(folium.Element(title))
map

(주)만성상사 대흥주유소
성원이앤에스(주)영등포지점


### 단계구분도

In [25]:
# 자치구 컬럼 만들기
lis = []
for i in df.index:
    lis.append(df.주소.loc[i].split()[1])
df['자치구'] = lis

In [27]:
# 구별 주유소 갯수 구하기
df_gu = df.groupby('자치구').count()['상호']

In [28]:
geo_data = json.load(open('skorea_municipalities_geo_simple.json', encoding='utf-8'))

In [29]:
# 지도
map = folium.Map(location=[37.581, 126.986], zoom_start=11, tiles='Stamen Toner')

folium.Choropleth(geo_data=geo_data, 
                  data=df_gu, 
                  columns=('자치구', '지역'), 
                  fill_color='YlOrRd', 
                  key_on='feature.id',
                  legended_name='주유소 수').add_to(map)
map

### 지도 완성

In [30]:
# 지도
map = folium.Map(location=[37.581, 126.986], zoom_start=11)

#단계구분도
folium.Choropleth(geo_data=geo_data, 
                  data=df_gu, 
                  columns=('자치구', '지역'), 
                  fill_color='YlOrRd', 
                  key_on='feature.id',
                  legended_name='주유소 수').add_to(map)
#장소 표시
for i in df.index:
    try:
        road_addr = df['주소'].loc[i]
        station_name = df['상호'].loc[i]
        station_type = df['상표'].loc[i]
        url = f'{search_url}?query={quote(road_addr)}'
        result = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}).json()
        lng = float(result['documents'][0]['x'])
        lat = float(result['documents'][0]['y'])
        folium.Marker(
            location = [lat, lng], 
            popup = folium.Popup(road_addr, max_width=200),
            tooltip = (station_name+f"({station_type})"), 
            icon = folium.Icon(color = 'black', icon = 'glyphicon glyphicon-tint')
        ).add_to(map)
    except:
        print(station_name)
title = "<h3 align = 'center' style = 'font-size:20px'> 서울자치구별 주유소 현황</h3>"
map.get_root().html.add_child(folium.Element(title))
map

(주)만성상사 대흥주유소
성원이앤에스(주)영등포지점


In [31]:
map.save('./서울주유소현황.html')